In [1]:
import pandas as pd
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go
from rich import print
from rich.progress import track
import json
import tools.smoothing as sm
import tools.ultrasound_algos as algos
import math
import us.utils as utils
import plotly.io as pio
pio.templates.default = "plotly"

In [2]:
def vline(fig, x, color="None", name=None):
    fig.add_scatter(x=[x, x], y=[0, 255], name=name, line_color=color, mode="lines", line_dash="dash")

In [3]:
# Agco silo infos
density = 65
agco_silo_data = {
    "ConeHeight": 2.918,
    "HeightGroundToBin": 0.7,
    "HeightGroundToTopOfCone": 3.618,
    "HeightGroundToTopOfCylinder": 6.895,
    "HeightGroundToTopOfBin": 8.230,
    "CylinderHeight": 3.277,
    "ConeAngle": 30,
    "RoofAngle": 50,
    "TopOfConeRadius": 1.82,
    "BottomOfConeRadius": 0.135,
    "TopOfRoofRadius": 0.229,
    "ConeVolume": 10.93,
    "BinVolume": 50.318,
    "SensorOffset": 0.335,
}

In [4]:
"""
0,5m = 48,821m3
1m = 45,124m3
2m = 34,718
5m = 5,055m3
"""
dist = 5
utils.dist_to_volume_agco(dist, agco_silo_data)

7.367641442431197

In [5]:
data_65 = pd.read_csv("../data/agco/v2/p2c-65.csv", converters={"AcquisitionTime": pd.to_datetime, "rawdata": json.loads})
data_9 = pd.read_csv("../data/agco/v2/p2c-9.csv", converters={"AcquisitionTime": pd.to_datetime, "rawdata": json.loads})
data_8 = pd.read_csv("../data/agco/v2/p2c-8.csv", converters={"AcquisitionTime": pd.to_datetime, "rawdata": json.loads})
data_17 = pd.read_csv("../data/agco/v2/p2c-17.csv", converters={"AcquisitionTime": pd.to_datetime, "rawdata": json.loads})
print(data_9.columns)

loadcells = pd.read_csv("../data/agco/v2/loadcells.csv", converters={"AcquisitionTime": pd.to_datetime})
print(loadcells.columns)
lc17 = loadcells[loadcells["LocationName"] == "MPass-10"]
lc9 = loadcells[loadcells["LocationName"] == "MPass-09"]
lc8 = loadcells[loadcells["LocationName"] == "MPass-07"]
lc65 = loadcells[loadcells["LocationName"] == "MPass-08"]

Index(['BatchId', 'AcquisitionTime', 'MetaData', 'UltrasoundData',
       'CorrelationId', 'SerialNumber', 'Hostname', 'temperature', 'CDM_index',
       'PGA_index', 'quality', 'config', 'device', 'pulseCount',
       'PGA_distance', 'PGA_weight', 'CDM_distance', 'CDM_weight', 'rawdata'],
      dtype='object')

Index(['AcquisitionTime', 'LocationName', 'LoadCellWeight_t'], dtype='object')

In [6]:
# Compute custom CDM on all dataset

In [7]:
# Compute lowpass and wf on all dataset
cutoff_freq = 150
for df in [data_65]:
    lowpass_weights = []
    computed_weights_before = []
    computed_weights_15 = []
    computed_weights_25 = []
    computed_weights_40 = []
    computed_weights_50 = []
    for i in track(range(len(df)), description=f"Computing {df.iloc[0]['device']}..."):
        raw_data = df.iloc[i]["rawdata"]
        temperature = df.iloc[i]["temperature"]
        pc = df.iloc[i]["pulseCount"]

        computed_wf = algos.before_wavefront(raw_data, temperature, 0.5, 10, pc, 0.1)
        computed_dist = utils.tof_to_dist2(computed_wf, temperature) * 2
        computed_weight = utils.dist_to_volume_agco(computed_dist, agco_silo_data) * density / 100
        computed_weights_before.append(computed_weight)

        computed_wf = algos.wavefront(raw_data, temperature, 0.15, 10, pc)
        computed_dist = utils.tof_to_dist2(computed_wf, temperature) * 2
        computed_weight = utils.dist_to_volume_agco(computed_dist, agco_silo_data) * density / 100
        computed_weights_15.append(computed_weight)

        computed_wf = algos.wavefront(raw_data, temperature, 0.25, 10, pc)
        computed_dist = utils.tof_to_dist2(computed_wf, temperature) * 2
        computed_weight = utils.dist_to_volume_agco(computed_dist, agco_silo_data) * density / 100
        computed_weights_25.append(computed_weight)

        computed_wf = algos.wavefront(raw_data, temperature, 0.40, 10, pc)
        computed_dist = utils.tof_to_dist2(computed_wf, temperature) * 2
        computed_weight = utils.dist_to_volume_agco(computed_dist, agco_silo_data) * density / 100
        computed_weights_40.append(computed_weight)

        computed_wf = algos.wavefront(raw_data, temperature, 0.50, 10, pc)
        computed_dist = utils.tof_to_dist2(computed_wf, temperature) * 2
        computed_weight = utils.dist_to_volume_agco(computed_dist, agco_silo_data) * density / 100
        computed_weights_50.append(computed_weight)

        b, a = signal.butter(2, cutoff_freq / 250000, 'lowpass', analog=False)
        lowpass = signal.filtfilt(b, a, raw_data)
        lowpass_wf = algos.wavefront(lowpass, temperature, 0.35, 10, pc)
        lowpass_dist = utils.tof_to_dist2(lowpass_wf, temperature) * 2
        lowpass_weight = utils.dist_to_volume_agco(lowpass_dist, agco_silo_data) * density / 100
        lowpass_weights.append(lowpass_weight)
    df["lowpass_weight"] = lowpass_weights
    df["computed_weight_15"] = computed_weights_15
    df["computed_weight_25"] = computed_weights_25
    df["computed_weight_40"] = computed_weights_40
    df["computed_weight_50"] = computed_weights_50
    df["computed_weight_before"] = computed_weights_before

Output()

In [8]:
# Match loadcells on original dataset

In [14]:

for df, lc, name in zip([data_65], [lc65], ["p2c-65"]):
    #fig = px.line(df, x="AcquisitionTime", y=["PGA_weight"], title=f"Silo: {name}", hover_data=[df.index])
    fig = go.Figure(layout_title=f"Silo: {name}", layout_height=400, layout_margin_t=70, layout_margin_b=0)
    fig.add_trace(go.Scatter(x=lc["AcquisitionTime"], y=lc["LoadCellWeight_t"], name="Loadcell", line_color="green", legendrank=3))
    fig.add_scatter(x=df["AcquisitionTime"], y=sm.smooth_all(df["AcquisitionTime"], df["lowpass_weight"], 40), name="New algo", legendrank=2)
    fig.add_scatter(x=df["AcquisitionTime"], y=df["PGA_weight"], line_color="gray", opacity=0.3, name="Original algo", legendrank=1)
    fig.add_scatter(x=df["AcquisitionTime"], y=df["computed_weight_50"], name="Original algo computed")
    fig.show()
#x = data_9["AcquisitionTime"]
#y = sm.smooth_all(data_9["AcquisitionTime"].values, data_9["PGA_weight"].values, 40)
#fig.add_trace(go.Scatter(y=y, name="Smoothed")).show()

#fig = px.line(data_65, x="AcquisitionTime", y=["PGA_weight", "CDM_weight", "lowpass_weight"], title="p2c-65", hover_data=[data_65.index])
#fig.add_trace(go.Scatter(x=lc17["AcquisitionTime"], y=lc17["LoadCellWeight_t"], name="loadcell"))
#fig.show()


In [28]:
indices = [135]
#indices = [240, 242, 245, 248, 250]
df = data_65
for i in indices:
    raw_data = df.iloc[i]["rawdata"]
    temperature = df.iloc[i]["temperature"]
    pc = df.iloc[i]["pulseCount"]

    cutoff_freq = 150
    b, a = signal.butter(2, cutoff_freq / 250000, 'lowpass', analog=False)
    lowpass = signal.filtfilt(b, a, raw_data)
    raw_wf = algos.wavefront(raw_data, temperature, 0.5, 10, pc)
    lowpass_wf = algos.wavefront(lowpass, temperature, 0.5, 10, pc)

    bang_end = algos.detect_main_bang_end(lowpass, pc)
    wf_before = algos.before_wavefront(lowpass, temperature, 0.5, 10, pc, 0.25)



    fig = go.Figure(layout_title=f"{i} - {df.iloc[i]['AcquisitionTime']}", layout_xaxis_title="Time", layout_yaxis_title="Signal")
    fig.add_scatter(y=raw_data, name="raw data")
    fig.add_scatter(y=lowpass, name="lowpass")
    vline(fig, raw_wf, color="blue", name="wavefront")
    vline(fig, lowpass_wf, color="red", name="wavefront lowpass")
    vline(fig, wf_before, color="yellow", name="wavefront before")
    vline(fig, bang_end, color="yellow", name="lowpass bang end")
    #cdm = algos.center_of_mass(raw_data, pc)
    #vline(fig, cdm, color="orange", name="cdm")
    fig.show()